# Dependencies 

In [3]:
import cv2 
import numpy as np 
import os 
import matplotlib.pyplot as plt 
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from keras.models import load_model

# Functions

In [4]:
def read_img(path):
    return cv.imread('11.jpg')

In [5]:
def rescal_frame(frame,dimensions):

    return cv2.resize(frame,dimensions,interpolation=cv2.INTER_AREA)

In [15]:
def text_put(frame,text):
    printed=f' Prediction is :{text}'
    org = (5, 20)
    fontScale = 0.5
    color = (255, 255, 255)
    thickness = 2
    font = cv2.FONT_HERSHEY_SIMPLEX

    #                   image  text      orgin  font_type
    image = cv2.putText(frame,  printed,    org,    font,
    #                font_scale,   color,   thickness  ,lineType  ,bottomLeftOrigin
                         fontScale, color, thickness,   cv2.LINE_AA, False)

    return frame

In [30]:
def get_pred(prediction):
    classes_name=['Glasses','No Glasses']

    return classes_name[np.argmax(prediction)]

# Face Detection Using Haar Cascades

## - Testing on Images

In [8]:
img = cv2.imread('11.jpg')
cv2.imshow('Male Face org',img)

gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
cv2.imshow('Male Face gray',gray)

har_file=cv2.CascadeClassifier('Har.xml')
face_rect=har_file.detectMultiScale(gray,scaleFactor=1.1,minNeighbors=1)
print(f"There are {len(face_rect)} person")


for (x,y,w,h) in face_rect:
    cv2.rectangle(img,(x,y) , (x+w, y+h) ,(255,0,0),thickness=3)
cv2.imshow('Detecyed Face org',img)


cv2.waitKey(15000)
cv2.destroyAllWindows()

There are 6 person


## - On Live Video

In [9]:
har_file=cv2.CascadeClassifier('Har.xml')


capture = cv2.VideoCapture(0)

while True:
    ret, frame=capture.read()
    frame=cv2.flip(frame,1)
    frame=rescal_frame(frame,(256,256))
    if not ret:
        break
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    gauss = cv2.GaussianBlur(gray, (21, 21), 0)
#     _, thresh = cv2.threshold(gauss, 100, 255, cv2.THRESH_BINARY_INV)
#     thresh= cv2.dilate(thresh,(3,3),2)
    
    face_rect=har_file.detectMultiScale(gauss,scaleFactor=1.1,minNeighbors=3)
    print(f"There are {len(face_rect)} person")
    for (x,y,w,h) in face_rect:
        cv2.rectangle(frame,(x,y) , (x+w, y+h) ,(0,255,0),thickness=3)
    frame=rescal_frame(frame,(728,512))
    cv2.imshow('Detecyed Face org',frame)
#     cv2.imshow('thresh',thresh)
#     cv2.imshow('gray',gray)
#     cv2.imshow('gauss',gauss)  
    
    if cv2.waitKey(20) & 0xFF==ord('d'):
        cv2.destroyAllWindows()
        break
capture.release()
        

There are 0 person
There are 1 person
There are 1 person
There are 1 person
There are 1 person
There are 1 person
There are 1 person
There are 1 person
There are 1 person
There are 1 person
There are 1 person
There are 1 person
There are 1 person
There are 1 person
There are 1 person
There are 1 person
There are 1 person
There are 1 person
There are 1 person
There are 1 person
There are 1 person
There are 1 person


# Model Loading

In [12]:
# model = load_model('modelMobnet3.h5')
# model = load_model('model_DA.h5')
model = load_model(r'C:\Users\user\Desktop\Project\Glasses and Faces Detection\Models\model_DL_fine_tuned.h5')
# model = load_model('Models/model_DA.h5')
# model = load_model('Models/model_DL_frozen.h5')

In [13]:
model.summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 7, 7, 1280)       2257984   
 ional)                                                          
                                                                 
 flatten_10 (Flatten)        (None, 62720)             0         
                                                                 
 dense_22 (Dense)            (None, 64)                4014144   
                                                                 
 dropout_10 (Dropout)        (None, 64)                0         
                                                                 
 dense_23 (Dense)            (None, 2)                 130       
                                                                 
Total params: 6,272,258
Trainable params: 5,875,714
Non-trainable params: 396,544
_____________________________________

# Face and Glasses Detection 

In [35]:
har_file=cv2.CascadeClassifier('Har.xml')


capture = cv2.VideoCapture(0)

while True:
    ret, frame=capture.read()
    if not ret:
        break
        
        
    frame=cv2.flip(frame,1)
    frame=rescal_frame(frame,(224,224))
    preprocessed_frame = preprocess_input(frame)

    copy=frame.copy()
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    gauss = cv2.GaussianBlur(gray, (21, 21), 0)
    
    
    face_rect=har_file.detectMultiScale(gauss,scaleFactor=1.1,minNeighbors=3)
    print(f"There are {len(face_rect)} person")
    for (x,y,w,h) in face_rect:
        cv2.rectangle(frame,(x-15,y-15) , (x+w+5, y+h+20) ,(0,255,0),thickness=3)
        
    croped=copy[y-35:y+h+35 ,x-35:x+w+35]
    croped_2=rescal_frame(copy,(256,256))
    preprocessed_frame_croped_2=rescal_frame(preprocessed_frame[y-20:y+h+20 ,x-5:x+w+35],(224,224))

    prediction=model.predict( np.expand_dims(preprocessed_frame_croped_2, axis=0) )
    
    out=get_pred(prediction)
    text_put(frame,out)
    print(out)



    

#     cv2.imshow('Detecyed Face org',rescal_frame(frame,(512,512)))
    cv2.imshow('Detecyed Face org',rescal_frame(frame,(512,512)))

#     cv2.imshow('gray',gray)
#     cv2.imshow('gauss',gauss)  
#     cv2.imshow('croped',croped_2)
    
    cv2.imshow('croped',croped)  
    
    
    
    if cv2.waitKey(20) & 0xFF==ord('d'):
        cv2.destroyAllWindows()
        break
capture.release()
        

There are 0 person
1/1 [==============================] - 0s 232ms/step
No Glasses
There are 0 person
1/1 [==============================] - 0s 230ms/step
Glasses
There are 0 person
1/1 [==============================] - 0s 210ms/step
Glasses
There are 1 person
1/1 [==============================] - 0s 206ms/step
Glasses
There are 1 person
1/1 [==============================] - 0s 217ms/step
No Glasses
There are 1 person
1/1 [==============================] - 0s 214ms/step
No Glasses
There are 1 person
1/1 [==============================] - 0s 222ms/step
Glasses
There are 1 person
1/1 [==============================] - 0s 221ms/step
Glasses
There are 1 person
1/1 [==============================] - 0s 212ms/step
Glasses
There are 1 person
1/1 [==============================] - 0s 219ms/step
No Glasses
There are 1 person
1/1 [==============================] - 0s 211ms/step
Glasses
There are 1 person
1/1 [==============================] - 0s 216ms/step
Glasses
There are 1 person
1/1 [====